In [1]:
%load_ext autoreload
%autoreload 2

import sys
import time
from functools import partial

# to get the ray dashboard, use `pip install 'ray[default]'`
import ray

sys.path.append('../')
from vflow import Vset, Vfunc, AsyncVfunc, init_args

In [2]:
# initialize ray
ray.init(num_cpus=2) # optionally might add _temp_dir='tmp'

2022-02-08 01:43:57,242	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '128.32.135.109',
 'raylet_ip_address': '128.32.135.109',
 'redis_address': '128.32.135.109:6379',
 'object_store_address': '/tmp/ray/session_2022-02-08_01-43-48_443597_1809522/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-02-08_01-43-48_443597_1809522/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-02-08_01-43-48_443597_1809522',
 'metrics_export_port': 54998,
 'node_id': '424cd56963cf8d269d8d1d186a9ddb07e1578a1dfe17815fca69c337'}

In [3]:
def slow_fun(a, b=1):
    time.sleep(1)
    return a + b


# create 4 vfuncs using partials of slow_fun
# the 'vfunc' arg to Vfunc.__init__ must either have a .fit method or be callable
vfuncs = [Vfunc(f'fun{i}', partial(slow_fun, b=i)) for i in range(4)]

slow_set = Vset('slow_set', vfuncs)

# if there's only one arg, it comes wrapped in a list and I can't pass the result of init_args directly to .fit
args = init_args([1], ['a'])[0]

In [4]:
%% time

slow_set(args)

UsageError: Cell magic `%%` not found.


In [5]:
# 1st option to use parallelism is to just use is_async=True when initializing VfuncSet
fast_set = Vset('fast_set', vfuncs, is_async=True)

In [6]:
%% time

fast_set(args)

UsageError: Cell magic `%%` not found.


In [7]:
# 2nd option is to use AsyncVfuncs directly
async_vfuncs = [AsyncVfunc(f'fun{i}', partial(slow_fun, b=i)) for i in range(4)]
fast_set = Vset('fast_set', async_vfuncs)

In [8]:
%% time

fast_set(args)

UsageError: Cell magic `%%` not found.
